# **WFLOW: DERIVING INPUT MAPS**

## *PACKAGES*


In [83]:
using  Revise, Rasters, GeoFormatTypes, GeoTIFF, ArchGDAL, Shapefile, NCDatasets

#  Geomorphometry
#  GeoDataFrames,
# using CairoMakie, Colors, ColorSchemes

Path_Home = @__DIR__
cd(Path_Home)

include("Julia//Parameters.jl")
include("Julia//GeoRaster.jl")
include("Julia//GeoNetcdf.jl")
include("Julia//PlotParameter.jl")
# include("Julia//GeoPlot.jl")

# For new projects if does not exist
   mkpath(joinpath(Path_Root, Path_InputForcing))
   mkpath(joinpath(Path_Root, Path_InputGis))
   mkpath(joinpath(Path_Root, Path_InputLookuptable))
   mkpath(joinpath(Path_Root, Path_NetCDF))
   mkpath(joinpath(Path_Root, Path_OutputJulia))
   mkpath(joinpath(Path_Root, Path_OutputPython))
   mkpath(joinpath(Path_Root, Path_OutputTimeSeriesWflow))
   mkpath(joinpath(Path_Root, Path_OutputWflow))

# Pkg.add(name="ArchGDAL", version="0.10.9")


"d:\\JOE\\MAIN\\MODELS\\WFLOW\\DATA\\Timoleague\\OutputWflow"

## **DEM OFFICIAL GRID SIZE**

	https://rafaqz.github.io/Rasters.jl/dev/api#Rasters.resample-Tuple
method: A Symbol or String specifying the method to use for resampling. From the docs for gdalwarp:

**:near:** nearest neighbour resampling (default, fastest algorithm, worst interpolation quality).
:bilinear: bilinear resampling.
:cubic: cubic resampling.
:cubicspline: cubic spline resampling.
:lanczos: Lanczos windowed sinc resampling.
:average: average resampling, computes the weighted average of all non-NODATA contributing pixels. rms root mean square / quadratic mean of all non-NODATA contributing pixels (GDAL >= 3.3)
:mode: mode resampling, selects the value which appears most often of all the sampled points.
:max: maximum resampling, selects the maximum value from all non-NODATA contributing pixels.
:min: minimum resampling, selects the minimum value from all non-NODATA contributing pixels.
:med: median resampling, selects the median value of all non-NODATA contributing pixels.
:q1: first quartile resampling, selects the first quartile value of all non-NODATA contributing pixels.
:q3: third quartile resampling, selects the third quartile value of all non-NODATA contributing pixels.
:sum: compute the weighted sum of all non-NODATA contributing pixels (since GDAL 3.1)

## **DEM OPERATIONS**

In [84]:
# include("Julia//GeoRaster.jl")
# Reading data
   Path_InputGis_Dem = joinpath(Path_Root, Path_InputGis, Dem_Input_Qgis)

   Dem_Resample₀ = Rasters.Raster(Path_InputGis_Dem)
   Dem_Resample₀ = Rasters.replace_missing(Dem_Resample₀, missingval=NaN)

   # Resample Dem in 2 steps
      Param_Crs_GeoFormat = GeoFormatTypes.convert(WellKnownText, EPSG(Param_Crs))

    #   Dem_Resample₀ = Rasters.resample(Dem_Resample₀; res=ΔX₁, method=ResampleMethod₁, crs=Param_Crs_GeoFormat)

      Dem_Resample₀  = Rasters.resample(Dem_Resample₀; res=ΔX₂, method=ResampleMethod₂, crs=Param_Crs_GeoFormat)

   # Writting the output
      Path_OutputJulia_Dem = joinpath(Path_Root, Path_OutputJulia, Dem_Julia)

      Rasters.write(Path_OutputJulia_Dem, Dem_Resample₀; ext=".tiff", missingval= NaN, force=true, verbose=true)

# Getting the metadata
	Metadatas₀ = geoRaster.RASTER_METADATA(Dem_Resample₀; Verbose=true)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Geomorphometry.multihillshade(Dem_Resample₀), Title="Digital Elevation Model", Label="Altitude [m]", colormap=:greys)
		geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Dem_Resample₀, Title="Digital Elevation Model", Label="Altitude [m]", colormap=:greys)
end


Param_Crs = 29902
ΔX = 10.0
ΔY = -10.0
N_Width  = 461
N_Height = 286
Coord_X_Left = 143100.001, Coord_X_Right = 147700.001
Coord_Y_Top = 42620.008, Coord_Y_Bottom = 39770.008


## ====== **PLEASE RUN PYTHON CODE** =======

++===========================================++


## **Reading the corrected DEM from python**

In [85]:
include("Julia//GeoRaster.jl")
# Reading data
   Path_InputPython_Dem = joinpath(Path_Root, Path_OutputPython, Dem_Input_Python)

   Dem_Resample₀ = Rasters.Raster(Path_InputPython_Dem)
   Dem_Resample  = Rasters.replace_missing(Dem_Resample₀, missingval=NaN)

# Writting the output
   Path_OutputJulia_Dem_Corrected = joinpath(Path_Root, Path_OutputJulia, Dem_Julia_Corrected)
   Rasters.write(Path_OutputJulia_Dem_Corrected, Dem_Resample; ext=".tiff", missingval= NaN, force=true, verbose=true)

# Getting the metadata
	Metadatas = geoRaster.RASTER_METADATA(Dem_Resample; Verbose=true)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Geomorphometry.multihillshade(Dem_Resample), Title="Corrected Digital Elevation Model", Label="Altitude [m]", colormap=:greys)
end


Param_Crs = 29902
ΔX = 10.0
ΔY = -10.0
N_Width  = 461
N_Height = 286
Coord_X_Left = 143100.001, Coord_X_Right = 147700.001
Coord_Y_Top = 42620.008, Coord_Y_Bottom = 39770.008


## **Longitute and Latitude**

In [86]:
include("Julia//GeoRaster.jl")

Longitude, Latitude = Rasters.X(Metadatas.Coord_X_Left: Metadatas.ΔX: Metadatas.Coord_X_Right, crs=Metadatas.Crs_GeoFormat), Rasters.Y(Metadatas.Coord_Y_Top: Metadatas.ΔY: Metadatas.Coord_Y_Bottom, crs=Metadatas.Crs_GeoFormat)

@assert(Metadatas.N_Width  == length(Longitude))
@assert(Metadatas.N_Height == length(Latitude))


## **CATCHMENT:** *Reading catchment boundary from Python used as a mask*

In [ ]:
# Reading Catchement map
   Path_InputPython_Subcatchment = joinpath(Path_Root, Path_OutputPython, Subcatch_Python)
   Subcatchment₀ = Rasters.Raster(Path_InputPython_Subcatchment)
   Subcatchment₁ = Rasters.replace_missing(Subcatchment₀, missingval=0)

# Resampling to the same grid size as the Dem_Resample
   # Subcatchment₁ = Rasters.resample(Subcatchment₀; to=Dem_Resample, missingval=0)

# Computing subcatchments
   Subcatchment = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat)

   for iX=1:Metadatas.N_Width
      for iY=1:Metadatas.N_Height
         if Subcatchment₁[iX,iY] > 0
            Subcatchment[iX,iY] = 1
         else
            Subcatchment[iX,iY] = 0
         end
      end # for iY=1:Metadatas.N_Height
   end # for iX=1:Metadatas.N_Width


## **GAUGE**

In [118]:
N_Width, N_Height  = size(Dem_Resample)

Gauge = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat)

X1= Param_GaugeCoordinate[1]
Y1 = Param_GaugeCoordinate[2]

A = Rasters.view(Gauge, X(X1..X1), Y(Y1..Y1))

println(A)

# iX, iY = LAT_LONG_2_INDEX(;Map=Gauge, Param_GaugeCoordinate)



0×0 view(::Matrix{Float64}, 362:361, 47:46) with eltype Float64


In [ ]:
include("Julia//GeoRaster.jl")

Path_InputGis_Gauge  = joinpath(Path_Root, Path_InputGis, Gauge_Shp)
println(Path_InputGis_Gauge)

# Load the shapefile
   Gauge_Raw = Shapefile.Handle(Path_InputGis_Gauge)

# Rasterizing
   Gauge = Rasters.rasterize(last, Gauge_Raw; res=Metadatas.ΔX, fill=1, missingval=0, crs=Metadatas.Crs_GeoFormat, to=Dem_Resample, boundary=:center, shape=:point ,progress=true)

# Saving file
   Path_OutputGauge = joinpath(Path_Root, Path_OutputWflow, Gauge_Wflow)
   Rasters.write(Path_OutputGauge, Gauge; ext=".tiff", force=true, verbose=true, missingval=0)

# Finding the Gauge coordinate
   iX_Gauge = 1
   iY_Gauge = 1
   for iX=1:Metadatas.N_Width
      for iY=1:Metadatas.N_Height
         if Gauge[iX, iY] == 1
            iX_Gauge = iX
            iY_Gauge = iY
            println( "Gauge:  iX= $iX_Gauge  iY= $iY_Gauge")
            break
         end
      end # for iY=1:Metadatas.N_Height
   end # for iX=1:Metadatas.N_Width

   # @assert(Subcatchment[iX_Gauge, iY_Gauge] == 1)

   iX₂, iY₂,= geoRaster.LAT_LONG_2_INDEX(;Map=Gauge, Param_GaugeCoordinate)

# Writting the output
   # Path_OutputWflow_Subcatchment = joinpath(Path_Root, Path_OutputWflow, Subcatch_Wflow)
   # Rasters.write(Path_OutputWflow_Subcatchment, Subcatchment; ext=".tiff", missingval=NaN, force=true, verbose=true)

# if 🎏_Plots
#    CairoMakie.activate!()
#    Fig_2 =  CairoMakie.Figure()
#    Axis_2 = CairoMakie.Axis(Fig_2[1, 1], title="Catchment Boundaries", xlabel= L"Longitude", ylabel=L"Latitude",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

#    Map_2 = CairoMakie.plot!(Axis_2, Subcatchment, colormap=Reverse(:greys))
#    Map_2 = CairoMakie.heatmap!(Axis_2, Gauge, colormap=:lajolla)

#    CairoMakie.display(Fig_2)
# end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\InputGis\Timoleague_Gauge_Hydro.shp
Gauge:  iX= 362  iY= 46
LAT_LONG_2_iCOORD: [362 ; 46]


(362, 46)

## **DEM:** *Mask to catchment area*

In [90]:
include("Julia//GeoRaster.jl")

Dem_Resample_Mask = geoRaster.MASK(;Input=Dem_Resample, Mask=Subcatchment, Param_Crs=Metadatas.Crs_GeoFormat, Latitude, Longitude)

🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Dem_Resample_Mask, Map₂=Subcatchment, Map₁_Nodata=NaN, Map₂_Nodata=0)

# Writting output
   Path_OutputJulia_DemMask = joinpath(Path_Root, Path_OutputJulia, Dem_Julia_Mask)

   Rasters.write(Path_OutputJulia_DemMask, Dem_Resample_Mask; ext=".tiff", missingval= NaN, force=true, verbose=true)

# Getting the metadata
	Metadatas = geoRaster.RASTER_METADATA(Dem_Resample_Mask; Verbose=true);

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Dem_Resample_Mask, Title="Digital Elevation Model Masked", Label="Altitude [m]", colormap=:greys)
end


Param_Crs = 29902
ΔX = 10.0
ΔY = -10.0
N_Width  = 461
N_Height = 286
Coord_X_Left = 143100.001, Coord_X_Right = 147700.001
Coord_Y_Top = 42620.008, Coord_Y_Bottom = 39770.008


## **DEM:** *flow accumulation*

In [91]:
if 🎏_Plots && 🎏_Plot_FlowAccumulation
   # Options: DInf(), D8(), FD8(2)
   Acc, Ldd = Geomorphometry.flowaccumulation(Dem_Resample; method=FD8(4))

   Acc_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Acc, Latitude, Longitude, Mask=Subcatchment)

	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= log10.(Acc_Mask), Title="Flow accumulation", Label="Flow accumulation [mm]", colormap=:rain)
end


## **LDD:** *Python Mask to catchment area* 

In [92]:
include("Julia//GeoRaster.jl")

Path_InputPython_Ldd = joinpath(Path_Root, Path_OutputPython, Ldd_Python)
Ldd₀                 = Rasters.Raster(Path_InputPython_Ldd)
Ldd₁                 = Rasters.replace_missing(Ldd₀, missingval=0)

# Resampling such that it has the same grid size as the Dem_Resample
   # Ldd₁ = Rasters.resample(Ldd₀; to=Dem_Resample_Mask, missingval=0) #

# Masking with catchment
   Ldd_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Ldd₁, Latitude, Longitude, Mask=Subcatchment, Missing=0)

# Writting output
   Path_OutputWflow_LddMask = joinpath(Path_Root, Path_OutputWflow, Ldd_Wflow)
   Rasters.write(Path_OutputWflow_LddMask, Ldd_Mask; ext=".tiff", force=true, verbose=true)
   println(Path_OutputWflow_LddMask)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Ldd_Mask, Title="Local Drain Direction", Label="Ldd [1-9]", colormap=Makie.Categorical(:Spectral_11))
end

🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Subcatchment, Map₂=Ldd_Mask,  Map₁_Nodata=0, Map₂_Nodata=0)


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\wflow_ldd.tiff


true

## **RIVER:** *Mask to catchment area*

In [93]:

Path_InputGis_River  = joinpath(Path_Root, Path_InputGis, River_Shp)
println(Path_InputGis_River)

# Load the shapefile
River_Raw = Shapefile.Handle(Path_InputGis_River)

River = Rasters.rasterize(last, River_Raw; res=ΔX₂, fill=1, missingval=0, crs=Metadatas.Crs_GeoFormat, to=Dem_Resample_Mask, shape=:line ,progress=true)

River_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=River, Latitude, Longitude, Mask=Subcatchment, Missing=0)

Path_OutputWflow_River = joinpath(Path_Root, Path_OutputWflow, River_Wflow)
Rasters.write(Path_OutputWflow_River, River_Mask; ext=".tiff", force=true, verbose=true)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_6 =  CairoMakie.Figure()
   # Axis_6 = CairoMakie.Axis(Fig_6[1, 1], title="Rivers", xlabel= L"$Latitude$", ylabel=L"$Longitude$", ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

   Map_6 = CairoMakie.plot!(Axis_6, River_Mask, colormap=Reverse(:lajolla))
   Map_6 = CairoMakie.heatmap!(Axis_6, Gauge, colormap=:lajolla)

   # CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
   CairoMakie.display(Fig_6)
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\InputGis\Timoleague_River3.shp


## **PITS**

In [94]:
Pits  = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat)

Pits_Count = 1
for iX=1:Metadatas.N_Width
   for iY=1:Metadatas.N_Height
      if Ldd₁[iX, iY] == 5
         Pits[iX,iY] = Pits_Count
         display("$iX, $iY")
         Pits_Count += 1
    else
        Pits[iX,iY] = NaN
      end
   end # for iY=1:Metadatas.N_Height
end # for iX=1:Metadatas.N_Width

# Writting output
   PathOutput_Pits = joinpath(Path_Root, Path_OutputJulia, Pits_Julia)
   Rasters.write(PathOutput_Pits, Pits; ext=".tiff", force=true, verbose=true)
   println(PathOutput_Pits)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_2 =  CairoMakie.Figure()
   Axis_2 = CairoMakie.Axis(Fig_2[1, 1], title="Catchment Boundaries", xlabel= L"Longitude", ylabel=L"Latitude",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

    CairoMakie.heatmap!(Axis_2, Dem_Resample, colormap=:greys)
    CairoMakie.heatmap!(Axis_2, Pits, colormap=Reverse(:greys))
    CairoMakie.heatmap!(Axis_2, RiverWidth, colormap=Reverse(:lajolla))
    # CairoMakie.heatmap!(Axis_2, Geomorphometry.multihillshade(Dem_Resample), colormap=:greys)
   # Map_2 = CairoMakie.heatmap!(Axis_2, Gauge, colormap=:lajolla)

   CairoMakie.display(Fig_2)
end


"1, 62"

"1, 74"

"1, 88"

"1, 99"

"2, 113"

...

"410, 217"

"411, 207"

"414, 192"

d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputJulia\Timoleague_Pits.tiff


LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LaTeXStrings in the current active module Main
in expression starting at d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X35sZmlsZQ==.jl:24

## **SLOPE:** *Reading Slope from Python & Mask*

In [95]:
Path_InputPython_Slope = joinpath(Path_Root, Path_OutputPython, Slope_Python)

Slope₀ = Rasters.Raster(Path_InputPython_Slope)

Slope = Rasters.replace_missing(Slope₀, missingval=NaN)

# Slope = Rasters.resample(Slope₀; to=Dem_Resample_Mask, missingval=0)

Slope_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Slope, Latitude, Longitude, Mask=Subcatchment)


# Writting the output
Path_OutputWflow_Slope = joinpath(Path_Root, Path_OutputWflow, RiverSlope_Wflow)
# Rasters.write(Path_OutputWflow_Slope, Slope_Mask; ext=".tiff", force=true, verbose=true)
println(Path_OutputWflow_Slope)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Slope_Mask, Title="Slope", Label="Slope", colormap=:matter)
end

# 🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Subcatchment, Map₂=Slope_Mask,  Map₁_Nodata=NaN, Map₂_Nodata=NaN)


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\RiverSlope.tiff


## **Rivers slope**

In [96]:
include("Julia//GeoRaster.jl")

RiverSlope = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Slope_Mask, Latitude, Longitude, Mask=River_Mask)

Path_OutputWflow_RiverSlope = joinpath(Path_Root, Path_OutputWflow, RiverSlope_Wflow)

Rasters.write(Path_OutputWflow_RiverSlope, RiverSlope; ext=".tiff", force=true, verbose=true)
println(Path_OutputWflow_RiverSlope)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_12 =  CairoMakie.Figure()
   Axis_12 = CairoMakie.Axis(Fig_12[1, 1], title="River Slope Per Pixel", xlabel= L"$Latitude$", ylabel=L"$Longitude$", ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize, titlecolor=titlecolor)

   # CairoMakie.plot!(Axis_12, River_Mask, colormap=Reverse(:lajolla))
   Map_12 = CairoMakie.heatmap!(Axis_12, RiverSlope, colormap=:viridis)

   Colorbar(Fig_12[1, 2], Map_12, label = "River slope [deg]", width = 15, ticksize = 15, tickalign = 0.5)
   display(Fig_12)
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\RiverSlope.tiff


LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LaTeXStrings in the current active module Main
in expression starting at d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X42sZmlsZQ==.jl:13

## **River width**

In [97]:
RiverWidth = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat, mappedcrs=EPSG(Metadatas.Param_Crs))
for iX=1:Metadatas.N_Width
   for iY=1:Metadatas.N_Height
      if River_Mask[iX,iY] == 1
         RiverWidth[iX,iY] = P_RiverWidth
      else
         RiverWidth[iX,iY] = NaN
      end
   end # for iY=1:Metadatas.N_Height
end # for iX=1:Metadatas.N_Width

Path_OutputWflow_RiverWidth = joinpath(Path_Root, Path_OutputWflow, RiverWidth_Wflow)
# Rasters.write(Path_OutputWflow_RiverWidth, RiverWidth; ext=".tiff", force=true, verbose=true)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_9 =  CairoMakie.Figure()
   Axis_9 = CairoMakie.Axis(Fig_9[1, 1], title="River Width", xlabel= L"$Latitude$", ylabel=L"$Longitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

   Map_9 = CairoMakie.plot!(Axis_9, RiverWidth, colormap=Reverse(:viridis))

   # CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
   CairoMakie.display(Fig_9)
end


LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LaTeXStrings in the current active module Main
in expression starting at d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X44sZmlsZQ==.jl:18

## **River depth**

In [98]:
RiverDepth = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat)

for iX=1:Metadatas.N_Width
   for iY=1:Metadatas.N_Height
      if River_Mask[iX,iY] == 1
         RiverDepth[iX,iY] = P_RiverDepth
      else
         RiverDepth[iX,iY] = NaN
      end
   end # for iY=1:Metadatas.N_Height
end # for iX=1:Metadatas.N_Width

Path_OutputWflow_RiverDepth = joinpath(Path_Root, Path_OutputWflow, RiverDepth_Wflow)
println(Path_OutputWflow_RiverDepth)

Rasters.write(Path_OutputWflow_RiverDepth, RiverDepth; ext=".tiff", force=true, verbose=true, crs=Metadatas.Crs_GeoFormat)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_10 =  CairoMakie.Figure()
   Axis_10 = CairoMakie.Axis(Fig_10[1, 1], title="River Depth", xlabel= L"$Latitudeitude$", ylabel=L"$Longitudegitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

   Map_10 = CairoMakie.plot!(Axis_10, RiverDepth, colormap=Reverse(:viridis))

   # CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
   CairoMakie.display(Fig_10)
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\wflow_riverdepth.tiff


LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LaTeXStrings in the current active module Main
in expression starting at d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X46sZmlsZQ==.jl:21

## **River length**

In [99]:
Path_InputPython_RiverLength = joinpath(Path_Root, Path_OutputPython, RiverLength_Python)
println(Path_InputPython_RiverLength)

RiverLength₀ = Rasters.Raster(Path_InputPython_RiverLength)
# RiverLength₁ = Rasters.resample(RiverLength₀; to=Dem_Resample_Mask, missingval=0)
RiverLength₁ = Rasters.replace_missing(RiverLength₀, missingval=NaN)

RiverLength_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=RiverLength₁, Latitude, Longitude, Mask=River_Mask)

Path_OutputWflow_RiverLength = joinpath(Path_Root, Path_OutputWflow, RiverLength_Wflow)
Rasters.write(Path_OutputWflow_RiverLength, RiverLength_Mask; ext=".tiff", force=true, verbose=true)
println(Path_OutputWflow_RiverLength)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_12 =  CairoMakie.Figure()
   Axis_12 = CairoMakie.Axis(Fig_12[1, 1], title="River Length Per Pixel", xlabel= L"$Latitudeitude$", ylabel=L"$Longitudegitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize, titlecolor=titlecolor)

   Map_12 = CairoMakie.heatmap!(Axis_12, RiverLength_Mask, colormap=:viridis)

   Colorbar(Fig_12[1, 2], Map_12, label = "River length [m]", width = 15, ticksize = 15, tickalign = 0.5)
   display(Fig_12)
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputPython\RiverLength.tiff
d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\wflow_riverlength.tiff


LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LaTeXStrings in the current active module Main
in expression starting at d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X51sZmlsZQ==.jl:17

## **Impermeable layers:**

In [100]:
if 🎏_ImpermeableMap
   Path_InputGis_Roads = joinpath(Path_Root, Path_InputGis, Roads_Shp)
   println(Path_InputGis_Roads)

   Road_Shapefile= GeoDataFrames.read(Path_InputGis_Roads)

   Impermeable = Rasters.rasterize(last, Road_Shapefile;  fill=1, res=Metadatas.ΔX, to=Dem_Resample_Mask, missingval=NaN, shape=:polygon, crs=Param_Crs_GeoFormat, boundary=:touches , progress=true, verbose=true)

   Impermeable_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Param_Crs_GeoFormat, Input=Impermeable, Latitude, Longitude, Mask=Subcatchment)

   Path_OutputWflow_Impermeable = joinpath(Path_Root, Path_OutputWflow, Impermable_Wflow)

   Rasters.write(Path_OutputWflow_Impermeable, Impermeable_Mask; ext=".tiff", force=true, verbose=true)

   geoPlot.HEATMAP(;Title="Roads", Label="Road", Input=Impermeable_Mask, 🎏_Colorbar=false)
else
   Impermeable_Mask = []
end


Any[]

# **SOIL MAPS**

In [101]:
if 🎏_SoilMap
   Path_InputSoils = joinpath(Path_Root, Path_InputGis, SoilMap_Raster)
   println(Path_InputSoils)

   SoilMapₒ = Rasters.Raster(Path_InputSoils)

   SoilMap₁ = Rasters.resample(SoilMapₒ; to=Dem_Resample_Mask, missingval=NaN)

   SoilMap = geoRaster.MASK(;Param_Crs=Metadatas.Param_Crs_GeoFormat, Input=SoilMap₁, Latitude, Longitude, Mask=Subcatchment)

   if 🎏_Plots
      geoPlot.HEATMAP(;🎏_Colorbar=true, Input=SoilMap, Label="Infiltration index [-]", Title ="Soil Maps" )
   end
end


# **LOOKUP TABLES**


## *Lookup hydro parameters*

In [102]:
if 🎏_SoilMap
   include("Julia//geoRaster.jl")
   include("Julia//Parameters.jl")

   Soil_Header, Soil_Maps = geoRaster.LOOKUPTABLE_2_MAPS(;🎏_Plots, Param_Crs=Metadatas.Param_Crs_GeoFormat, Dem_Resample_Mask, Latitude, Longitude, LookupTable=Lookup_Hydro, Map_Shp=SoilMap_Shp, Map_Value=:Drainage_C, Metadatas, Path_InputGis, Path_Root, Path_Root_LookupTable, Subcatchment, TitleMap="Soil Map", ΔX=ΔX₂)
else
   Soil_Header = Soil_Maps = []
end # if 🎏_SoilMap


Any[]

## *Lookup vegetation parameters*

In [103]:
   include("Julia//geoRaster.jl")
   include("Julia//Parameters.jl")
if 🎏_VegetationMap
   Vegetation_Header, Vegetation_Maps = geoRaster.LOOKUPTABLE_2_MAPS(;🎏_Plots, Colormap=:plasma, Param_Crs=Metadatas.Param_Crs_GeoFormat, Dem_Resample_Mask, Latitude, Longitude, LookupTable=Lookup_Vegetation, Map_Shp=VegetationMap_Shp, Map_Value=:CROP_DESC, Metadatas, Path_InputGis, Path_Root, Path_Root_LookupTable, Subcatchment, TitleMap="Vegetation Map", ΔX=ΔX₂)
else
   Vegetation_Header = Vegetation_Maps = []

end # if 🎏_VegetationMap


Any[]

# **NETCDF**

## ***Static maps***

In [104]:
include("Julia//GeoNetcdf.jl")

if 🎏_NetCDF
   NetCDF, Path_NetCDF_Full = geoNetcdf.TIFF_2_NETCDF(Gauge, Impermable_Wflow, Impermeable_Mask, Latitude, Ldd_Mask, Longitude, Metadatas, River_Mask, River_Wflow, RiverDepth, RiverDepth_Wflow, RiverLength_Mask, RiverSlope, RiverSlope_Wflow, RiverWidth, RiverWidth_Wflow, Slope_Mask, Soil_Header, Soil_Maps, Subcatch_Wflow, Subcatchment, Vegetation_Header, Vegetation_Maps)
   end


Base.IOError: IOError: unlink("D:\\JOE\\MAIN\\MODELS\\WFLOW\\Wflow.jl\\Wflow\\Data\\input\\Timoleague\\staticmaps-Timoleague.nc"): resource busy or locked (EBUSY)

In [105]:
Output_NCDatasets = NCDatasets.NCDataset(Path_NetCDF_Full)


Dataset: D:\JOE\MAIN\MODELS\WFLOW\Wflow.jl\Wflow\Data\input\Timoleague\staticmaps-Timoleague.nc
Group: /

Dimensions
   x = 461
   y = 286

Variables
  x   (461)
    Datatype:    Float64 (Float64)
    Dimensions:  x
    Attributes:
     units                = m
     comments             = lon

  y   (286)
    Datatype:    Union{Missing, Float64} (Float64)
    Dimensions:  y
    Attributes:
     _FillValue           = NaN
     units                = m
     comments             = lat

  lon   (461)
    Datatype:    Union{Missing, Float64} (Float64)
    Dimensions:  x
    Attributes:
     _FillValue           = NaN
     units                = m
     comments             = lon

  lat   (286)
    Datatype:    Union{Missing, Float64} (Float64)
    Dimensions:  y
    Attributes:
     _FillValue           = NaN
     units                = m
     comments             = lat

  wflow_ldd   (461 × 286)
    Datatype:    Union{Missing, UInt8} (UInt8)
    Dimensions:  x × y
    Attributes:
     _Fill

In [106]:
NCDatasets.close(Output_NCDatasets)


closed Dataset

### ***Plotting NetCDF maps***

In [107]:
if 🎏_Plots && 🎏_Plot_NetCDF
	using NCDatasets

	Path_NetCDF_Full  = joinpath(Path_Root_NetCDF, NetCDF_Instates)
	Output_NCDatasets = NCDatasets.NCDataset(Path_NetCDF_Full)

	Keys = NCDatasets.keys(Output_NCDatasets)

	Keys_Select = []
	for iiKeys ∈ Keys
		Output = Output_NCDatasets[iiKeys]
		Dimensions = length(size(Output))

		if Dimensions == 2
				push!(Keys_Select, String(iiKeys))
		end # Dimensions == 2
	end # for iiKeys ∈ Keys

	Keys_Select = String.(Keys_Select)
	println(Keys_Select)

	for iiKeys ∈ Keys_Select
		println(iiKeys)
		Map₀ = Array(Output_NCDatasets[iiKeys])

		# Clean data
			FillValue = Output_NCDatasets[iiKeys].attrib["_FillValue"]
			Map =  replace(Map₀, FillValue => NaN)

		geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Map, Label="$iiKeys", Title="$iiKeys", Xlabel= L"$iX$", Ylabel=L"$iY$", Yreversed=true, colormap=:hawaii50)
	end
end  # if: 🎏_Plots &&

Output_NCDatasets = nothing


LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LaTeXStrings in the current active module Main
in expression starting at d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y103sZmlsZQ==.jl:30

## ***Timeseries NetCDF maps***

In [108]:
if 🎏_TimeSeries
	include("Parameters.jl")
	include("GeoNetcdf.jl")
   include("GeoPlot.jl")

	NetCDFmeteo, Path_NetCDFmeteo_Output = geoNetcdf.TIMESERIES_2_NETCDF(Latitude, Longitude, Metadatas, Subcatchment)

	if 🎏_Plot_TimeSeries
   	# precip pet temp
   	geoPlot.HEATMAP_TIME(;Path=Path_NetCDFmeteo_Output, NameOutput="temp", Layer=1)
	end
end



SystemError: SystemError: opening file "d:\\JOE\\MAIN\\MODELS\\WFLOW\\WflowDataJoe\\WflowRaster.jl\\src\\Parameters.jl": No such file or directory

In [ ]:

# include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\GeoRaster.jl")

# Path_InputGis_Gauge  = joinpath(Path_Root, Path_InputGis, Gauge_Shp)
# println(Path_InputGis_Gauge)

# # Load the shapefile
# Gauge_Raw = Shapefile.Handle(Path_InputGis_Gauge)

# Gauge = Rasters.rasterize(last, Gauge_Raw; res=ΔX₂, fill=1, missingval=NaN, crs=Metadatas.Param_Crs_GeoFormat, to=Dem_Resample_Mask, shape=:point ,progress=true)

# for iLongitudeg=1:Metadatas.N_Width
# 	for iLatitude=1:1:Metadatas.N_Height
# 		if Gauge[iLongitudeg, iLatitude] == 1

#          println( "Point:  Nlongitude= $(Metadatas.N_Width) iLongitudegitude= $iLongitudeg Nlatitude= $(Metadatas.N_Height) iLatitudeitude= $iLatitude")

# 		end
# 	end # for iY=1:Metadatas.N_Height
# end # for iX=1:Metadatas.N_Width

# iLatitude, iLongitudeg, Latitudeitude, Longitudegitude, Nlatitude, Nlongitude = geoRaster.LAT_LONG_2_INDEX(;Map=Gauge, Param_GaugeCoordinate)

# Path_OutputJulia_Gauge = joinpath(Path_Root, Path_OutputJulia, Gauge_Julia)
# Rasters.write(Path_OutputJulia_Gauge, Gauge; ext=".tiff", force=true, verbose=true, missingval= NaN)

# if 🎏_Plots
#    using CairoMakie, Colors, ColorSchemes
#    include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\GeoPlot.jl")

#    CairoMakie.activate!()
#    Fig_6 =  CairoMakie.Figure()
#    Axis_6 = CairoMakie.Axis(Fig_6[1, 1], title="Rivers", xlabel= L"$Latitudeitude$", ylabel=L"$Longitudegitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

#    Map_6 =  CairoMakie.heatmap!(Axis_6, Gauge, colormap=:lajolla)

#    # CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
#    CairoMakie.display(Fig_6)
# end



In [110]:
# Path = raw"D:\JOE\MAIN\MODELS\WFLOW\DATA\Ballycanew\InputGis\Ballycanew_DTM_1M.tif"
# Dem_Resample₀ = Rasters.Raster(Path, missingval=NaN)
# Dem_Resample₁ = Rasters.resample(Dem_Resample₀; res=5.0, method=ResampleMethod₁, crs=Param_Crs_GeoFormat)
# Dem_Resample  = Rasters.resample(Dem_Resample₁; res=10.0, method=ResampleMethod₂, crs=Param_Crs_GeoFormat)

# 	Path_Output = raw"D:\JOE\MAIN\MODELS\WFLOW\DATA\Ballycanew\OutputJulia\FlowAccumulation.tif"
# 	Rasters.write(Path_Output, log10.(Acc); ext=".tiff", missingval= NaN, force=true, verbose=true)


In [111]:
# Path_OutputJulia_DemMask = joinpath(Path_Root, Path_OutputJulia, Dem_Julia_Mask)

# Dem_Resample_Mask₂ = Rasters.Raster(Path_OutputJulia_DemMask)

# # Dem_Resample_Mask₂ = Rasters.replace_missing(Dem_Resample_Mask₂, missingval=NaN)

# Geomorphometry.cellsize(Dem_Resample)

# # Dem_FillDepression = Geomorphometry.filldepressions(Dem_Resample)
# Acc, Ldd₁ = Geomorphometry.flowaccumulation(Dem_Resample_Mask₂; method=D8(), cellsize = [Metadatas.ΔX, Metadatas.ΔY])

# Ldd_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Param_Crs_GeoFormat, Input=Ldd₁, Latitude, Longitude, Mask=Subcatchment, Missing=0)

# if 🎏_Plots
# 	geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Ldd_Mask, Title="Local Drain Direction", Label="Ldd [1-9]", colormap=Makie.Categorical(:Spectral_11))
# end

# 🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Subcatchment, Map₂=Ldd_Mask,  Map₁_Nodata=0, Map₂_Nodata=0)
